# SBML to ODE for autoReduce
7.14.2020 <br>
Ankita Roychoudhury


In [1]:
# imports
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

**What do we want?** <br>
Input a SBML file, output `f`. <br>
A `f` object that is made up of sympy equations.

We will use the generated code function, similar to before. Except we will only return the array of functions and will not plot. We will also add the line `x,f,P = system.load_ODE_model(num_species, num_params)` so `f` can be created with sympy object.

In [2]:

def generateCodeForFile(filename,  t0=0, tEnd=10, numPoints=1000):
  # 
  # read the SBML from file 
  # 
  doc = readSBMLFromFile(filename)
  if doc.getNumErrors(LIBSBML_SEV_FATAL):
    print('Encountered serious errors while reading file')
    print(doc.getErrorLog().toString())
    sys.exit(1)
    
  # clear errors
  doc.getErrorLog().clearLog()
  
  #
  # perform conversions
  #
  props = ConversionProperties()
  props.addOption("promoteLocalParameters", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandInitialAssignments", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandFunctionDefinitions", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  #
  # figure out which species are variable 
  #
  mod = doc.getModel()
  variables = {}
#  print('params', mod.getNumParameters())

  for i in range(mod.getNumSpecies()): 
     species = mod.getSpecies(i)
    # if species.getBoundaryCondition() == True or variables.has_key(species.getId()):
     if species.getBoundaryCondition() == True or (species.getId() in variables.keys()):


       continue
     variables[species.getId()] = []
  #print(variables)
  
  #
  # start generating the code by appending to bytearray
  #
  # Define num_species, num_params here, will save and be used
  num_species = mod.getNumSpecies()
  num_params = mod.getNumParameters()
    
    
  generated_code = bytearray()
  generated_code.extend('from numpy import *\n'.encode('latin-1'))
  generated_code.extend('x,f,P = system.load_ODE_model(num_species, num_params)'.encode('latin-1'))


 # generated_code.extend('from matplotlib.pylab import *\n'.encode('latin-1'))
 # generated_code.extend('from matplotlib.pyplot import *\n'.encode('latin-1'))
 # generated_code.extend('from scipy.integrate import odeint \n'.encode('latin-1'))
  
  generated_code.extend('\n'.encode('latin-1'))
#  generated_code.extend('def simulateModel(t0, tend, numPoints):\n'.encode('latin-1'))
  
  # write out compartment values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#compartments\n'.encode('latin-1'))
  for i in range(mod.getNumCompartments()):
    element = mod.getCompartment(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getSize()).encode('latin-1'))
  
  # write out parameter values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#global parameters\n'.encode('latin-1'))
  for i in range(mod.getNumParameters()):
    element = mod.getParameter(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getValue()).encode('latin-1'))
  
  # write out boundary species 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#boundary species\n'.encode('latin-1'))
  for i in range(mod.getNumSpecies()):
    element = mod.getSpecies(i)
    if element.getBoundaryCondition() == False: 
      continue 
    if element.isSetInitialConcentration(): 
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialConcentration()).encode('latin-1'))
    else:
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialAmount()).encode('latin-1'))  
  
  # write derive function
  generated_code.extend('\n'.encode('latin-1'))
  #generated_code.extend('  def ode_fun(__Y__, t):\n'.encode('latin-1'))
    
   # Set all species equal to a __Y__[i] 
  for i in range(len(variables.keys())): 
    generated_code.extend('{0} = x[{1}]\n'.format(list(variables.keys())[i], i).encode('latin-1'))
    
  generated_code.extend('\n'.encode('latin-1'))
  
  for i in range(mod.getNumReactions()): 
    reaction = mod.getReaction(i)
    kinetics = reaction.getKineticLaw()  
    
    generated_code.extend('{0} = {1}\n'.format(reaction.getId(),  kinetics.getFormula()).encode('latin-1'))
    
    for j in range(reaction.getNumReactants()): 
      ref = reaction.getReactant(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('-{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('-({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
    for j in range(reaction.getNumProducts()): 
      ref = reaction.getProduct(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
  
  generated_code.extend('\n'.encode('latin-1'))
    
  generated_code.extend('funcs = array(['.encode('latin-1'))
 # generated_code.extend('    print( array(['.encode('latin-1'))
  for i in range(len(variables.keys())):
    for eqn in variables[list(variables.keys())[i]]:
      generated_code.extend(' + ({0})'.format(eqn).encode('latin-1'))
    if i + 1 < len(variables.keys()):
      generated_code.extend(',\n      '.encode('latin-1'))
  generated_code.extend('    ])\n'.encode('latin-1'))
  generated_code.extend('\n'.encode('latin-1'))

  return generated_code

In [4]:
filename = 'CRN.xml'

In [14]:
# Define num_species, num_params
doc = readSBMLFromFile(filename)
props = ConversionProperties()
props.addOption("promoteLocalParameters", True)
doc.convert(props)
mod = doc.getModel()

num_species = mod.getNumSpecies()
num_params = mod.getNumParameters()

In [17]:
gen_code = generateCodeForFile(filename)
exec(gen_code)

In [19]:
#returned in variable funcs
funcs[0]

-15.0*x0*x1*x2 - 1.5*x0*x3*x4 + 1.5*x5 + 15.0*x6

In [1]:
from sympy import Symbol
x1 = Symbol(element.getId())

NameError: name 'element' is not defined

In [22]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,auto_reduce,sympy,libsbml,sys,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
auto_reduce 0.1
sympy 1.5.1
libsbml 5.18.0
sys 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
jupyterlab 1.2.6
